This Notebook evaluates a convergence study for the 1D Shock-Acoustic-Wave interaction problems as presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [2]:
#r "BoSSSpad.dll"
#r "XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [3]:
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D_ConvergenceStudy2");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 15; Grid Count = 15; Path = \\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D_ConvergenceStudy2 }

In [4]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-05-05 16:14:45Z

In [5]:
var sessions = database.Sessions;
sessions

#0: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:17:01 PM	70a6e44f...
#1: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:14:32 PM	e3f53a43...
#2: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:45 PM	0db8d5ad...
#3: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:16:45 PM	08534bdf...
#4: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:14:14 PM	54ce6b78...
#5: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:31 PM	f2364944...
#6: XESTSF_AcousticWave1D	AW_p3_xCells31_tCells31_tMax4_1sinus_M

In [6]:
var aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

70a6e44f-fa41-46e6-be25-980d7fcdc570

## Choose which runs to plot

In [24]:
var sessions=database.Sessions.Where(s => (s.Name.Contains("sP1.5") &&s.Name.Contains("ampneg0")&&s.Name.Contains("amppos1")&& !s.Name.Contains("241"))); // the one plotted in the publication
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


#0: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:45 PM	0db8d5ad...
#1: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:31 PM	f2364944...
#2: XESTSF_AcousticWave1D	AW_p3_xCells31_tCells31_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:15 PM	7c29fe91...
#3: XESTSF_AcousticWave1D	AW_p3_xCells16_tCells16_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:01 PM	570b6f7f...


# A Helper Function to obtain different plot formats

In [25]:
using BoSSS.Solution.Tecplot;
using System.IO;
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Residual History

In [26]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 0 
 
 
 
 
 10 
 
 
 
 
 20 
 
 
 
 
 30 
 
 
 
 
 40 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - AWp3xCells121tCells121tMax1.51sinusMach1.5sP1.5wP0ampneg0amppos1E-05wLTruewSTruewL0.8 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M53.9,86.9 L67.8,287.2 L81.7,373.7 L95.7,376.4 L109.6,377.7 L123.5,378.8 L137.4,381.3 L151.4,382.3
 L165.3,382.8 L179.2,382.7 L193.1,383.8 L207.1,385.5 L221.0,385.6 L234.9,386.0 L248.8,386.0 L262.8,386.1
 L276.7,386.0 L290.6,386.0 L304.5,386.2 L318.5,386.2 L332.4,386.2 L346.3,269.5 L360.2,310.4 L374.2,341.2
 L388.1,364.0 L402.0,365.5 L415.9,365.9 L429.8,366.9 L443.8,367.6 L457.7,370.7 L471.6,369.9 L485.5,265.8
 L499.5,267.9 L513.4,285.4 L527.3,287.7 L541.2,287.8 L555.2,287.9 L569.1,287.9 L583.0,287.9 L596.9,287.9
 L610.9,287.9 L624.8,251.0 L638.7,254.9 L652.6,265.4 L666.6,269.5 L680.5,276.0 L694.4,279.4 L708.3,281.0
 L722.3,281.1 L736.2,281.2 L750.1,281.3 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResAWp3xCells121tCells121tMax1.51sinusMach1.5sP1.5wP0ampneg0amppos1E-05wLTruewSTruewL0.8 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M53.9,76.7 L67.8,77.3 L81.7,77.3 L95.7,77.3 L109.6,77.3 L123.5,77.3 L137.4,77.3 L151.4,77.3
 L165.3,77.3 L179.2,77.3 L193.1,77.3 L207.1,77.3 L221.0,77.3 L234.9,77.3 L248.8,77.3 L262.8,77.3
 L276.7,77.3 L290.6,77.3 L304.5,77.3 L318.5,77.3 L332.4,77.3 L346.3,88.2 L360.2,88.2 L374.2,88.2
 L388.1,88.2 L402.0,88.2 L415.9,88.2 L429.8,88.2 L443.8,88.2 L457.7,88.2 L471.6,88.2 L485.5,121.3
 L499.5,121.3 L513.4,121.3 L527.3,121.3 L541.2,121.3 L555.2,121.3 L569.1,121.3 L583.0,121.3 L596.9,121.3
 L610.9,121.3 L624.8,133.1 L638.7,133.1 L652.6,133.1 L666.6,133.1 L680.5,133.1 L694.4,133.1 L708.3,133.1
 L722.3,133.1 L736.2,133.1 L750.1,133.1 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - AWp3xCells61tCells61tMax1.51sinusMach1.5sP1.5wP0ampneg0amppos1E-05wLTruewSTruewL0.8 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M53.9,92.0 L67.8,299.0 L81.7,389.2 L95.7,389.8 L109.6,392.1 L123.5,392.3 L137.4,395.1 L151.4,394.5
 L165.3,395.3 L179.2,397.0 L193.1,396.6 L207.1,397.1 L221.0,397.3 L234.9,397.5 L248.8,397.6 L262.8,397.6
 L276.7,397.6 L290.6,397.6 L304.5,397.6 L318.5,397.6 L332.4,275.2 L346.3,375.4 L360.2,377.0 L374.2,377.6
 L388.1,377.8 L402.0,379.9 L415.9,381.7 L429.8,382.1 L443.8,382.4 L457.7,382.4 L471.6,290.7 L485.5,368.6
 L499.5,370.1 L513.4,370.8 L527.3,371.1 L541.2,371.9 L555.2,371.9 L569.1,371.9 L583.0,371.4 L596.9,371.2
 L610.9,278.1 L624.8,342.7 L638.7,362.0 L652.6,362.1 L666.6,363.6 L680.5,363.0 L694.4,364.6 L708.3,365.5
 L722.3,365.2 L736.2,364.7 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResAWp3xCells61tCells61tMax1.51sinusMach1.5sP1.5wP0ampneg0amppos1E-05wLTruewSTruewL0.8 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M53.9,81.7 L67.8,82.3 L81.7,82.3 L95.7,82.3 L109.6,82.3 L123.5,82.3 L137.4,82.3 L151.4,82.3
 L165.3,82.3 L179.2,82.3 L193.1,82.3 L207.1,82.3 L221.0,82.3 L234.9,82.3 L248.8,82.3 L262.8,82.3
 L276.7,82.3 L290.6,82.3 L304.5,82.3 L318.5,82.3 L332.4,84.1 L346.3,84.1 L360.2,84.1 L374.2,84.1
 L388.1,84.1 L402.0,84.1 L415.9,84.1 L429.8,84.1 L443.8,84.1 L457.7,84.1 L471.6,110.9 L485.5,110.9
 L499.5,110.9 L513.4,110.9 L527.3,110.9 L541.2,110.9 L555.2,110.9 L569.1,110.9 L583.0,110.9 L596.9,110.9
 L610.9,124.1 L624.8,124.1 L638.7,124.1 L652.6,124.1 L666.6,124.1 L680.5,124.

helper function to evaluate the XDG field along a x-ray for a specific time

In [27]:
(double[] tVals, double[]yVals) GetPlotAlongXRay(double tMin,double tMax, double x, int nRef,string field,ITimestepInfo ti){
//double shockPosition=0.5;
var p1= new double[] {x,tMin};
var p2=new double[] {x,tMax};

//timesteps to compare

// # use p_per from BoSSS 
var pEnd =ti.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] tVals= GenericBlas.Linspace(tMin,tMax,nRef);
return(tVals,yVals);
}


helper function to evaluate the XDG field along a t-ray for a point x

In [28]:
(double[] xVals, double[]yVals) GetPlotAlongTRay(double xMin,double xMax, double t, int nRef,string field,ITimestepInfo ti){
//double shockPosition=0.5;
var p1= new double[] {xMin,t};
var p2=new double[] {xMax,t};

//timesteps to compare

// # use p_per from BoSSS 
var pEnd =ti.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

helper function to compute the error metrics (i.e. the deviation of each amplitude from the exact amplification/reduction factor)

In [29]:
List<double> ComputeAmpErrors(double tMin,double tMax, double xMin, double xMax, int xRef, int tRef,string field,ITimestepInfo ti, bool isMax,double exact_amp_fac){
    if(true){
        var xVals= GenericBlas.Linspace(xMin,xMax,xRef);
        var maxminVals = new List<double>();
        foreach(var x in xVals ){
            (double[] tVals, double[] yVals)=GetPlotAlongXRay(tMin,tMax,x,tRef,field,ti);
            if(isMax){
                maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Max())/exact_amp_fac));
            }else{
                maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Min())/exact_amp_fac));
            }
        }
        return maxminVals;
    }else{
        var pEnd =ti.GetField(field);
        var fullcm= CellMask.GetFullMask(pEnd.GridDat);
        var ba= new bool[fullcm.ItemEnum.Count()];
        
        var bb = new BoSSS.Platform.Utils.Geom.BoundingBox(new double[,] {{0,0},{1,1}});
        foreach(var cell in fullcm.ItemEnum){
            pEnd.GridDat.iGeomCells.GetCellBoundingBox(cell, bb);
            var min = bb.Min;
            if(!isMax){
                if( min[0]>=xMin && min[0]<=xMax){
                    ba[cell]=true;
                };
            }else{
                if( min[0]>=xMin && min[0]<=xMax ){
                    ba[cell]=true;
                };
            }

        }
        var cm =new CellMask(pEnd.GridDat,new BitArray(ba));
        var valMin=new double[cm.ItemEnum.Count()];
        var valMax=new double[cm.ItemEnum.Count()];
        pEnd.GetCellwiseExtremalValues(valMin, valMax,cm);
        List<double> maxminVals;
        if(isMax){
            maxminVals=new List<double> {Math.Abs((exact_amp_fac-valMax.Max())/exact_amp_fac)};
        }else{
            maxminVals=new List<double> {Math.Abs((exact_amp_fac-valMin.Min())/exact_amp_fac)};
        }
        return maxminVals;
    }
    
}

var eps=0.000001;
double xMin=1.8 +eps;double xMax=2.0-eps; int xRef=5; 
double tMin =0 + eps;int tRef=800;
GenericBlas.Linspace(xMin,xMax,xRef)

[ 1.800001, 1.8500005, 1.9, 1.9499995, 1.999999 ]

In [30]:
var ses= sessions;
ses

#0: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:45 PM	0db8d5ad...
#1: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:31 PM	f2364944...
#2: XESTSF_AcousticWave1D	AW_p3_xCells31_tCells31_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:15 PM	7c29fe91...
#3: XESTSF_AcousticWave1D	AW_p3_xCells16_tCells16_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:01 PM	570b6f7f...


choose the sessions

they must be reordered for the plot to work...

In [31]:
var ses= sessions;
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

#0: XESTSF_AcousticWave1D	AW_p3_xCells16_tCells16_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:01 PM	570b6f7f...
#1: XESTSF_AcousticWave1D	AW_p3_xCells31_tCells31_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:15 PM	7c29fe91...
#2: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:31 PM	f2364944...
#3: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/5/2024 4:15:45 PM	0db8d5ad...


Number of cells

In [32]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

[ 256, 961, 3721, 14641 ]

helper arrays

In [33]:
int[] degs = new int[] {0,1,2,3};
//degs = new int[] {0};
var errors_array= MultidimensionalArray.Create(degs.Count(),ses.Count());


Function used to calculate the EOCs

In [34]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

Function used to compute the ideal convergence plot

In [35]:
List<double> GetIdealConvPlot(List<double> errors,List<double> cellSize, int p){
        var idealErrors = new List<double>();

    // Calculate ideal errors for each cell size
    for (int i = 0; i < cellSize.Count; i++)
    {
        double idealError = errors[0] * Math.Pow(cellSize[i] / cellSize[0], p + 1);
        idealErrors.Add(idealError);
    }

    return idealErrors;
} 

compute the exact analytical pressure amplification/reduction factor for both cases investigated

In [36]:
// Given variables
var MBaseL = 1.5; // Base Mach number on the left
var gamma = 1.4; //Specific heat ratio

// Calculate right Mach number (Mach_R)
var MBaseR  = Math.Sqrt((1 + ((gamma-1)/2) * MBaseL*MBaseL) / (gamma * MBaseL*MBaseL - (gamma-1)/2));

// Upstream fast acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrL = ((1+MBaseL)*(1+MBaseL)) / (1+2*MBaseR+1/MBaseL/MBaseL) * (1 - ((gamma-1)/(gamma+1)) * (1-(1/MBaseL))* (1-(1/MBaseL)));

// Downstream slow acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrMiR = -(1- 2*MBaseR +1 / MBaseL/MBaseL) / (1+ 2*MBaseR +1 / MBaseL/ MBaseL);
(delta_pPrR_over_delta_pPrL,delta_pPrR_over_delta_pPrMiR)

Item1,2.154925878735875
Item2,-0.01484811194806477


Create the convergence plot

In [37]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
MultidimensionalArray errorsAndCells=MultidimensionalArray.Create(ses.Count(),degs.Length +1);
MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(ses.Count(),degs.Length+1);
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        Console.Write($"P={p},sess={iSess}");
        //extract the relevant field and timestep Index
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            cells.Add(Math.Sqrt(fields[0].Basis.GridDat.iGeomCells.GetCellVolume(0)));
            //extract AmpError
            // if(si.Name.Contains("sP0.5")){
            //     double exact_amp_fac=delta_pPrR_over_delta_pPrMiR*1e-05; double tMin=0.0 +eps;double tMax=4.0-eps;
            //     List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p_per",lts,false,exact_amp_fac);
            //     //errors_array[p,iSess]=ampErrors.Average();
            //     errors.Add(ampErrors.Average());
            //     Console.WriteLine($" sp0.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}");
            // }else{
            //     double exact_amp_fac=delta_pPrR_over_delta_pPrL*1e-05; double tMin=0.0 +eps;double tMax=1.0-eps;
            //     List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p_per",lts,true,exact_amp_fac);
            //     //errors_array[p,iSess]=ampErrors.Average();
            //     errors.Add(ampErrors.Average());
            //     Console.WriteLine($" sp1.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}");
            // }
             if(si.Name.Contains("sP0.5")){

                double t=2.5;double xMin=0.6;double xMax=2.999; int xRef=800;
                (double[] xVals_dummy, double[] yVals) = GetPlotAlongTRay(xMin,xMax,t,xRef,"p_per",lts);
                var ex_amp=delta_pPrR_over_delta_pPrMiR*1e-5;
                errors.Add(Math.Abs(yVals.Min()-ex_amp)/ex_amp);
                Console.WriteLine($" sp0.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}, amp:{yVals.Min()}");
                
            }else{

                double t=0.8;double xMin=1.6;double xMax=2.999; int xRef=800;
                (double[] xVals_dummy, double[] yVals) = GetPlotAlongTRay(xMin,xMax,t,xRef,"p_per",lts);
                var ex_amp=delta_pPrR_over_delta_pPrL*1e-5;
                errors.Add(Math.Abs(yVals.Max()- ex_amp)/ ex_amp);
                Console.WriteLine($" sp1.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}, amp:{yVals.Max()}");
                
            }
            
        }
    }
    errorsAndCells.SetColumn(0,cells);
    errorsAndCellsIdeal.SetColumn(0,cells);
    errorsAndCells.SetColumn(p+1,errors);
    errorsAndCellsIdeal.SetColumn(p+1,GetIdealConvPlot(errors,cells,p));
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    var fmt = GetFormat(count);
    fmt.PointSize=0.01;
    fmt.DashType=DashTypes.Dashed;

    plot.AddDataGroup($"ideal {p+1}",cells,GetIdealConvPlot(errors,cells,p), fmt);
    count++;
}
errorsAndCells.SaveToTextFile("ConvergenceStudyErrors.txt");
errorsAndCellsIdeal.SaveToTextFile("ConvergenceStudyErrorsIdeal.txt");
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 
 ideal 1 
 
 
 ideal 1 
 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 ideal 2 
 
 
 ideal 2 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 ideal 3 
 
 
 ideal 3 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3 
 
 
 
 
 
 
 
 
 
 
 ideal 4 
 
 
 ideal 4

In [38]:
EOCs

[ 0.06286044915210959, 1.368842266552513, 3.2255133641055136, 4.974562409869032 ]

Assertion

In [ ]:
NUnit.Framework.Assert.IsTrue(EOCs[0]>0.01);

In [ ]:
NUnit.Framework.Assert.IsTrue(EOCs[1]>1.2);

In [ ]:

NUnit.Framework.Assert.IsTrue(EOCs[2]>2.9;

In [ ]:
NUnit.Framework.Assert.IsTrue(EOCs[3]>3.9);

plot it

In [314]:
// using BoSSS.Solution.Tecplot;
// using System.IO;
// Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
// IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
// //double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
// var allTs = (List<double>) lts.TimeStepNumbers; 
// var N=40;
// var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
// foreach(var si in ses){
//     foreach(var timestep in si.Timesteps){
//         if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
//                 IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
//                 var SF = lts.GetShadowFields();
//                 var texplot = new Tecplot(timestep.Fields.Pick(0).GridDat, 2);
//                 texplot.PlotFields( si.Name + "_" + lts.TimeStepNumber,0.0,timestep.Fields);
//         }
//     }
// }